# Environment setup

The `scooby doo` dataset can be downloaded from Kaggle https://www.kaggle.com/datasets/williamschooleman/scoobydoo-complete <br>
Once the snowflake environment is set (creation of snowflake objects such as roles, DB, schema, WH), this code stages the dataset from a local file and then uses snowpark to write the clean data to a snowflake table. <br>
<br>
To setup your python environment and assuming you have conda or miniconda installed, you can create the environment using the conda_env.yml requirements.

`conda env create -f conda_env.yml`

Once it is created, you can activate it by calling:

`conda activate snowpark-ml-scooby`


**Note:** One step of the preparation uses the function `array_sort` introduced in `snowflake-snowpark-python==1.6.1` <br>
This might be ahead of the version in Snowflake instance, so we might receive some warning regarding this.

## Imports

In [ ]:
#Snowpark for python
from snowflake.snowpark.session import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StructType, StructField, FloatType, StringType, IntegerType, BooleanType, DateType
import snowflake.snowpark.functions as F

#Data Science libs
import numpy as np

#Misc 
import getpass
import pandas as pd
import json

## Create Snowpark Session

In [ ]:
accountname = getpass.getpass() # ORGNAME-ACCOUNTNAME (separated by minus sign)

In [ ]:
username = getpass.getpass()    # SNOWFLAKE-USERNAME

In [ ]:
password = getpass.getpass()    # SNOWFLAKE-PASSWORD

In [ ]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "LAB001_ROLE",
    "database" : 'LAB001_DB',
    "schema" : 'SCOOBY_SCHEMA',
    "warehouse" : 'LAB001_WH',
    "ENABLE_ARRAY_SORT_FUNCTION" : True
}

session = Session.builder.configs(connection_parameters).create()


snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

## Save credentials to a JSON file for reuse
Store the connection_parameters into a JSON file for reuse later in the HOL.  
**PLEASE BE AWARE THAT PASSWORD ARE STORED IN CLEAR TEXT!**

In [ ]:
with open('creds.json', 'w') as outfile:
        outfile.write(json.dumps(connection_parameters))

Create the Snowpark session with saved credentials

In [ ]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

session = Session.builder.configs(connection_parameters).create()

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

## Data Ingestion
### Use the Snowpark DataFrame Reader to read in data from the local CSV file 

For more information on loading data, see documentation on [snowflake.snowpark.DataFrameReader](https://docs.snowflake.com/ko/developer-guide/snowpark/reference/python/api/snowflake.snowpark.DataFrameReader.html).

In [ ]:
# Stage the local file
_ = session.file.put("../data/scoobydoo_20211019.csv", "@LAB001_DB.SCOOBY_SCHEMA.SCOOBY_ASSETS ", auto_compress=False)

In [ ]:
# Define the schema for the data in the CSV file, all string as we'll cast later on
scooby_schema = StructType([StructField("index", StringType()), 
                              StructField("series_name", StringType()),
                              StructField("network", StringType()),
                              StructField("season", StringType()), 
                              StructField("title", StringType()),
                              StructField("imdb", StringType()),
                              StructField("engagement", StringType()),
                              StructField("date_aired", StringType()),
                              StructField("run_time", StringType()),
                              StructField("format", StringType()),
                              StructField("monster_name", StringType()),
                              StructField("monster_gender", StringType()),
                              StructField("monster_type", StringType()),
                              StructField("monster_subtype", StringType()),
                              StructField("monster_species", StringType()),
                              StructField("monster_real", StringType()),
                              StructField("monster_amount", StringType()),
                              StructField("caught_fred", StringType()),
                              StructField("caught_daphne", StringType()),
                              StructField("caught_velma", StringType()),
                              StructField("caught_shaggy", StringType()),
                              StructField("caught_scooby", StringType()),
                              StructField("captured_fred", StringType()),
                              StructField("captured_daphne", StringType()),
                              StructField("captured_velma", StringType()),
                              StructField("captured_shaggy", StringType()),
                              StructField("captured_scooby", StringType()),
                              StructField("unmask_fred", StringType()),
                              StructField("unmask_daphne", StringType()),
                              StructField("unmask_velma", StringType()),
                              StructField("unmask_shaggy", StringType()),
                              StructField("unmask_scooby", StringType()),
                              StructField("snack_fred", StringType()),
                              StructField("snack_daphne", StringType()),
                              StructField("snack_velma", StringType()),
                              StructField("snack_shaggy", StringType()),
                              StructField("snack_scooby", StringType()),
                              StructField("unmask_other", StringType()),
                              StructField("caught_other", StringType()),
                              StructField("caught_not", StringType()),
                              StructField("trap_work_first", StringType()),
                              StructField("setting_terrain", StringType()),
                              StructField("setting_country_state", StringType()),
                              StructField("suspects_amount", StringType()),
                              StructField("non_suspect", StringType()),
                              StructField("arrested", StringType()),
                              StructField("culprit_name", StringType()),
                              StructField("culprit_gender", StringType()),
                              StructField("culprit_amount", StringType()),
                              StructField("motive", StringType()),
                              StructField("if_it_wasnt_for", StringType()),
                              StructField("and_that", StringType()),
                              StructField("door_gag", StringType()),
                              StructField("nunber_of_snacks", StringType()),
                              StructField("split_up", StringType()),
                              StructField("another_mystery", StringType()),
                              StructField("set_a_trap", StringType()),
                              StructField("jeepers", StringType()),
                              StructField("jinkies", StringType()),
                              StructField("my_glasses", StringType()),
                              StructField("just_about_wrapped_up", StringType()),
                              StructField("zoinks", StringType()),
                              StructField("groovy", StringType()),
                              StructField("scooby_doo_where_are_you", StringType()),
                              StructField("rooby_rooby_roo", StringType()),
                              StructField("batman", StringType()),
                              StructField("scooby_dum", StringType()),
                              StructField("scrappy_doo", StringType()),
                              StructField("hex_girls", StringType()),
                              StructField("blue_falcon", StringType()),
                              StructField("fred_va", StringType()),
                              StructField("daphne_va", StringType()),
                              StructField("velma_va", StringType()),
                              StructField("shaggy_va", StringType()),
                              StructField("scooby_va", StringType())
                              ])

# Create a Snowpark DataFrame that is configured to load data from the CSV file
scooby_df = session.read.options({"field_delimiter": ",", 
                                    "skip_header": 1,
                                    "field_optionally_enclosed_by": '"'}).schema(scooby_schema).csv("@SCOOBY_ASSETS")

Cast the data types per type, use try_cast that fills with NULL if there is an invaid value

In [ ]:
int_array = ["ENGAGEMENT","RUN_TIME","MONSTER_AMOUNT","SUSPECTS_AMOUNT","CULPRIT_AMOUNT","SPLIT_UP","ANOTHER_MYSTERY","SET_A_TRAP","JEEPERS","JINKIES","MY_GLASSES"
,"JUST_ABOUT_WRAPPED_UP","ZOINKS","GROOVY","SCOOBY_DOO_WHERE_ARE_YOU","ROOBY_ROOBY_ROO"]
float_array = ["IMDB"]
boolean_array = ["MONSTER_REAL","CAUGHT_FRED","CAUGHT_DAPHNE","CAUGHT_VELMA","CAUGHT_SHAGGY","CAUGHT_SCOOBY"
,"CAPTURED_FRED","CAPTURED_DAPHNE","CAPTURED_VELMA","CAPTURED_SHAGGY","CAPTURED_SCOOBY"
,"UNMASK_FRED","UNMASK_DAPHNE","UNMASK_VELMA","UNMASK_SHAGGY","UNMASK_SCOOBY"
,"SNACK_FRED","SNACK_DAPHNE","SNACK_VELMA","SNACK_SHAGGY","SNACK_SCOOBY","UNMASK_OTHER","CAUGHT_OTHER","CAUGHT_NOT","TRAP_WORK_FIRST","NON_SUSPECT","ARRESTED","DOOR_GAG"
,"BATMAN","SCOOBY_DUM","SCRAPPY_DOO","HEX_GIRLS","BLUE_FALCON"]

for col in int_array :
    scooby_df = scooby_df.with_column(col, F.try_cast(col,IntegerType()))

for col in float_array:
    scooby_df = scooby_df.with_column(col, F.try_cast(col,FloatType()))

for col in boolean_array:
    scooby_df = scooby_df.with_column(col, F.try_cast(col,BooleanType()))

for col in ["DATE_AIRED"]:
    scooby_df = scooby_df.with_column("DATE_AIRED_PARSED", F.try_cast(col,DateType()))


scooby_df.show()
print(scooby_df.dtypes)

In [ ]:
# Look at descriptive stats on the DataFrame
scooby_df.describe().show()

### Exploratory data analysis

Explore the categorical columns that are considered candidate for the ML prediction

In [ ]:
# Categorical Columns 
cat_array = ["FORMAT","NETWORK","SETTING_TERRAIN","SETTING_COUNTRY_STATE","MOTIVE"
             ,"MONSTER_GENDER","MONSTER_TYPE","MONSTER_SUBTYPE","MONSTER_SPECIES","CULPRIT_GENDER"]

for c in cat_array:
    print(c + " " + str(len(set(scooby_df.select(F.col(c)).collect()))))

print(scooby_df.count())
print(set(scooby_df.select(F.col("FORMAT")).collect()))
print(set(scooby_df.select(F.col("NETWORK")).collect()))
print(set(scooby_df.select(F.col("SETTING_TERRAIN")).collect()))
print(set(scooby_df.select(F.col("SETTING_COUNTRY_STATE")).collect()))
print(set(scooby_df.select(F.col("MOTIVE")).collect()))


print(set(scooby_df.select(F.col("MONSTER_GENDER")).collect()))
print(set(scooby_df.select(F.col("MONSTER_TYPE")).collect()))
print(set(scooby_df.select(F.col("MONSTER_SUBTYPE")).collect()))
print(set(scooby_df.select(F.col("MONSTER_SPECIES")).collect()))
print(set(scooby_df.select(F.col("CULPRIT_GENDER")).collect()))



In [ ]:
#Few categories contain - and empty space , so we'll substitute this in the cleaning to make Super-Villain and Sea-Monster SUPERVILLAIN and SEAMONSTER and Possessed Object to POSSESEDOBJECT
scooby_df.filter(F.contains(F.col("MONSTER_TYPE"),F.lit(" ")) | F.contains(F.col("MONSTER_TYPE"),F.lit("-"))).show()

## Data Cleaning

According to the findings of EDA, clean the data

### 1. Standardise category formatting

First, we standardize the category formatting for  `FORMAT` , `NETWORK` , `SETTING_TERRAIN` , `SETTING_COUNTRY_STATE` , `MOTIVE` using Snowpark DataFrame operations.

This way, when we write to a Snowflake table, there will be no inconsistencies in how the Snowpark DataFrame will read in the column names. Secondly, the feature transformations on categoricals will be easier to encode.

In [ ]:
unique_cat_array = ["FORMAT","NETWORK","SETTING_TERRAIN","SETTING_COUNTRY_STATE","MOTIVE"]

def fix_values(column):
    return F.upper(F.regexp_replace(F.col(column), '[^a-zA-Z0-9]+', '_'))

for col in unique_cat_array:
    scooby_df = scooby_df.with_column(col, fix_values(col))

scooby_df.show()

### 2. Standardised categories that contain various values 

When MONSTER_AMOUNT > 1 or CULPRIT_AMOUNT > 1, the columns contain one value per monster or culprit <br>
For monster we standardised: `MONSTER_GENDER`,`MONSTER_TYPE`,`MONSTER_SUBTYPE`,`MONSTER_SPECIES` <br>
For culprit we standardise: `CULPRIT_GENDER` <br>
To standardised:
<li> Clean individual value: upper cased, right trim for some values that contain a comma (,) at the end, <br> 
delete all non letters or number characters (except comma) using regexp_replace and empty substitute </li>
<li> Transform the string to an array, deduple values and sort in alphabetical order </li>
<li> Standardised to convert all non-letter and non-numbers characters to _ </li>

In [ ]:
monster_cat_array = ["MONSTER_GENDER","MONSTER_TYPE","MONSTER_SUBTYPE","MONSTER_SPECIES"]
culprit_cat_array = ["CULPRIT_GENDER"]
# [^a-zA-Z0-9|,]+   \-+
def unique_array(column):
    ar = F.array_to_string(
        F.array_sort(F.array_distinct(F.strtok_to_array(
            F.regexp_replace(F.rtrim(F.upper(F.col(column)),F.lit(",")), '[^a-zA-Z0-9|,]+', '')
            ,F.lit(","))))
        ,F.lit(","))
    return F.iff(F.col(column) != 'NULL', ar , None)

for col in monster_cat_array:
    scooby_df = scooby_df.with_column(col, F.iff(scooby_df["MONSTER_AMOUNT"]>1,unique_array(col),
    F.regexp_replace(F.rtrim(F.upper(F.col(col)),F.lit(",")), '[^a-zA-Z0-9|,]+', '')))

for col in culprit_cat_array:
    scooby_df = scooby_df.with_column(col, F.iff(scooby_df["CULPRIT_AMOUNT"]>1,unique_array(col),
    F.regexp_replace(F.rtrim(F.upper(F.col(col)),F.lit(",")), '[^a-zA-Z0-9|,]+', '')))

for col in monster_cat_array:
    scooby_df = scooby_df.with_column(col, fix_values(col))

for col in culprit_cat_array:
    scooby_df = scooby_df.with_column(col, fix_values(col))

scooby_df.show()

### 3. Review effects of the cleaning

In [ ]:
# Categorical Columns after data cleaning
cat_array = ["FORMAT","NETWORK","SETTING_TERRAIN","SETTING_COUNTRY_STATE","MOTIVE"
             ,"MONSTER_GENDER","MONSTER_TYPE","MONSTER_SUBTYPE","MONSTER_SPECIES","CULPRIT_GENDER"]

for c in cat_array:
    print(c + " " + str(len(set(scooby_df.select(F.col(c)).collect()))))

print(scooby_df.count())
print(set(scooby_df.select(F.col("FORMAT")).collect()))
print(set(scooby_df.select(F.col("NETWORK")).collect()))
print(set(scooby_df.select(F.col("SETTING_TERRAIN")).collect()))
print(set(scooby_df.select(F.col("SETTING_COUNTRY_STATE")).collect()))
print(set(scooby_df.select(F.col("MOTIVE")).collect()))


print(set(scooby_df.select(F.col("MONSTER_GENDER")).collect()))
print(set(scooby_df.select(F.col("MONSTER_TYPE")).collect()))
print(set(scooby_df.select(F.col("MONSTER_SUBTYPE")).collect()))
print(set(scooby_df.select(F.col("MONSTER_SPECIES")).collect()))
print(set(scooby_df.select(F.col("CULPRIT_GENDER")).collect()))

In [ ]:
scooby_df.filter(F.col("MONSTER_AMOUNT")>1).show()

In [ ]:
scooby_df.filter(F.contains(F.col("MONSTER_TYPE"),F.lit("POSSES"))).show()

In [ ]:
# Look at descriptive stats on the DataFrame
scooby_df.describe().show()

### 4. Write cleaned data to a snowflake table

In [ ]:
scooby_df.write.mode('overwrite').save_as_table('scooby_clean')

In [ ]:
session.close()